# This notebook is used for modelling between the livability of melbourne and rental affordability

For the sake of simplicty, this notebook contains the following
- Livability scores of melbourne
- Rental affordability of greater Melbourne

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm

## Data processing

The below data will be used for this model:

### Melbourne livability score

- This dataset contains multiple liveability related scores for multiple regions

### Predictor variable: Greater Melbourne rental affordability index

- This dataset contains information about the rental affordability 

In [89]:
# Load data

liveability = pd.read_csv("../data/predictor/Melbourne Liveability Indicators.csv")
rental_affordability = pd.read_csv("../data/response/rental_affordability.csv")
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')

In [90]:
LIVEABILITY_COLUMNS = []

for column in liveability.columns:
    LIVEABILITY_COLUMNS.append(column)

In [91]:
LIVEABILITY_COLUMNS

['type',
 'topic',
 'id',
 'indicator',
 'period',
 'numerator',
 'denominator',
 'value',
 'value_type',
 'sources']

In [92]:
set(liveability['period'])

{'2006',
 '2011',
 '2012',
 '2013',
 '2013/14',
 '2014',
 '2014 (June Qtr)',
 '2014 (at 30 June)',
 '2014/15',
 '2015',
 '2015 (June Qtr)',
 '2015 (at 30 June)',
 '2015/16',
 '2016',
 '2016 (June Qtr)',
 '2016 (at 30 June)',
 '2016/17',
 '2017',
 '2017 (June Qtr)',
 '2017 (at 30 June)',
 '2018',
 '2018 (June Qtr)',
 '2018 (at 30 June)',
 '2019',
 'FY 2013/14',
 'FY 2014/15',
 'FY 2015/16',
 'FY 2016/17',
 'FY 2017/18',
 'FY 202013/14',
 'FY 202014/15',
 'FY 202015/16'}

For the liveability dataset, we have information from 2006, 2011~2019. Now we proceed to look at rental_affordability

In [93]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
5,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


In [94]:
rental_affordability.columns.sort_values()

Index(['city', 'geography_name', 'rai_cityadjusted_total_2011_q1',
       'rai_cityadjusted_total_2011_q2', 'rai_cityadjusted_total_2011_q3',
       'rai_cityadjusted_total_2011_q4', 'rai_cityadjusted_total_2012_q1',
       'rai_cityadjusted_total_2012_q2', 'rai_cityadjusted_total_2012_q3',
       'rai_cityadjusted_total_2012_q4', 'rai_cityadjusted_total_2013_q1',
       'rai_cityadjusted_total_2013_q2', 'rai_cityadjusted_total_2013_q3',
       'rai_cityadjusted_total_2013_q4', 'rai_cityadjusted_total_2014_q1',
       'rai_cityadjusted_total_2014_q2', 'rai_cityadjusted_total_2014_q3',
       'rai_cityadjusted_total_2014_q4', 'rai_cityadjusted_total_2015_q1',
       'rai_cityadjusted_total_2015_q2', 'rai_cityadjusted_total_2015_q3',
       'rai_cityadjusted_total_2015_q4', 'rai_cityadjusted_total_2016_q1',
       'rai_cityadjusted_total_2016_q2', 'rai_cityadjusted_total_2016_q3',
       'rai_cityadjusted_total_2016_q4', 'rai_cityadjusted_total_2017_q1',
       'rai_cityadjusted_total_20

From a quick look , it looks like geography_name=3005 seems to only hold NaN values, and thus can get rid of it

In [95]:
rental_affordability[rental_affordability['geography_name']==3005]

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
rental_affordability[['unique_id', 'city', 'geography_name', 'state']]

,unique_id,city,geography_name,state
0,1310,Greater Melbourne,3000,VIC
1,1312,Greater Melbourne,3003,VIC
2,1313,Greater Melbourne,3004,VIC
3,1314,Greater Melbourne,3005,VIC
4,1315,Greater Melbourne,3006,VIC
5,1316,Greater Melbourne,3008,VIC


From the above information, we can remove all columns but geography_name (which seems to be postcode), and also remove where geography_name=3005

In [97]:
rental_affordability = rental_affordability.drop(3)
rental_affordability = rental_affordability.reset_index()
rental_affordability.drop(columns=rental_affordability.columns[0], axis=1, inplace=True)

Here, despite the wide range of information, we only need the estimated population data (erp_YYYY) for our model. Further, notice the random spacing before each column name, asides from the first.

In [76]:
# Get all key columns for our response dataset. The years range from 2001 to 2021, inclusive of start & end. 

KEY_RESPONSE_COLUMNS = ['lga_name_2021']

for i in range(2001, 2022):
    key_column = f"erp_{i}"
    KEY_RESPONSE_COLUMNS.append(key_column)

In [88]:
response[KEY_RESPONSE_COLUMNS]

,lga_name_2021,erp_2001,erp_2002,erp_2003,erp_2004,erp_2005,erp_2006,erp_2007,erp_2008,erp_2009,...,erp_2012,erp_2013,erp_2014,erp_2015,erp_2016,erp_2017,erp_2018,erp_2019,erp_2020,erp_2021
0,Banyule,117967,117559,117313,116661,117149,117963,119362,120580,122248,...,123584,124314,125107,126088,127447,129115,130250,131640,131895,129387
1,Bayside (Vic.),88262,88717,89088,89394,89873,90807,92041,93535,94850,...,96971,98252,99797,101268,102912,104282,105745,106856,107516,105580
2,Boroondara,156246,156886,157298,157885,158727,159605,161308,163408,165284,...,168483,170485,172660,174753,177276,179677,181376,183197,182974,176632
3,Darebin,127067,127359,127711,128735,130298,132299,134748,137525,140289,...,144986,147272,149511,151974,155126,158751,161653,164224,166356,162501
4,Glen Eira,122346,123197,123816,125037,126485,128474,130524,132900,135414,...,139116,141260,143694,145924,148583,151457,153920,156535,158149,155123
5,Hobsons Bay,82854,83124,83107,82668,82990,83906,84819,85513,86767,...,88339,89188,90417,91674,93445,95071,96481,97765,98146,96317
6,Maribyrnong,60848,61557,62516,63168,64137,65518,67666,69902,71872,...,77179,79782,82463,84783,86942,89362,91413,93482,94928,93467
7,Melbourne,55398,60415,65954,71425,76034,80154,85366,89937,94247,...,107578,118711,127975,136872,146096,159141,170317,178994,183628,169860
8,Moonee Valley,104793,104856,104629,104632,105059,105717,106716,108417,110367,...,113673,115464,117763,120231,122871,125411,127906,130313,131702,129379
9,Moreland,135540,136156,136630,137642,138963,140872,143694,146450,150114,...,156967,160282,164099,167870,172294,176974,181780,185811,188660,184707


Now, we separate this dataframe into a format we're able to use

In [180]:
# Pick out row 7 to turn into dataframe

response_clean = pd.DataFrame(response[KEY_RESPONSE_COLUMNS].iloc[7]).reset_index()

In [181]:
# Rename and clean columns

response_clean = response_clean.rename(columns={'index': 'year', 7: 'population'})
response_clean = response_clean.iloc[1:] # Remove unnceessary row
response_clean['year'] = response_clean['year'].str.removeprefix('erp_')
response_clean['year'] = response_clean['year'].astype(int)

Now we move onto our predictor dataset to have a quick look

In [68]:
len(predictor)

63568

In [36]:
predictor.columns

Index(['council_ref', 'permit_number', 'issue_date', 'address',
       'desc_of_works', 'estimated_cost_of_works', 'rbs_number',
       'commence_by_date', 'completed_by_date', 'permit_certificate_type',
       'year'],
      dtype='object')

Note that this dataset has gotten an initial clean, removing records where:
- issue_date > completed_by_date
- Year is in the future

However, there seems to be records of duplicate permits (same coucil_ref). We observe more closely

In [56]:
predictor_count = predictor.groupby(['council_ref'])['permit_number'].count().sort_values(ascending=False).reset_index(name='count')
predictor_count[predictor_count['count'] > 1]

,council_ref,count
0,BP-2019-1273,66
1,BP-2020-1544,56
2,BP-2020-460/4,42
3,BP-2018-2091,36
4,BP-2019-1273/1,33
...,...,...
19315,BP-2016-2542,2
19316,BP-2016-2392/8,2
19317,BP-2016-242,2
19318,BP-2023-1058,2


Here, observe a few highest values and see why it's been duplicated

In [59]:
predictor[predictor['council_ref'] == 'BP-2019-1273']

,council_ref,permit_number,issue_date,address,desc_of_works,estimated_cost_of_works,rbs_number,commence_by_date,completed_by_date,permit_certificate_type,year
2993,BP-2019-1273,1585/20190205/1A,1559001600000,Part Level 5 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019
2994,BP-2019-1273,1585/20190205/1A,1559001600000,Level 9 276 Flinders Street MELBOURNE VIC 3000,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019
2995,BP-2019-1273,1585/20190205/1A,1559001600000,Level 6 276 Flinders Street MELBOURNE VIC 3000,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019
2996,BP-2019-1273,1585/20190205/1A,1559001600000,Part Level 9 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019
2997,BP-2019-1273,1585/20190205/1A,1569888000000,Part Level 8 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Certificate of Final Inspection,2019
...,...,...,...,...,...,...,...,...,...,...,...
111939,BP-2019-1273,1585/20190205/1A,1563148800000,Part Level 7 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Certificate of Final Inspection,2019
111940,BP-2019-1273,1585/20190205/0,1559001600000,Part Level 8 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019
111941,BP-2019-1273,1585/20190205/1A,1563148800000,Level 8 276 Flinders Street MELBOURNE VIC 3000,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Certificate of Final Inspection,2019
111942,BP-2019-1273,1585/20190205/1A,1559001600000,Part Level 8 276 Flinders Street MELBOURNE VIC...,Proposed Fitout of Levels 5-9 for the Purpose ...,2922588,None,2020-05-28,2022-05-28,Building Permit,2019


In [62]:
predictor[predictor['council_ref'] == 'BP-2020-1544']

,council_ref,permit_number,issue_date,address,desc_of_works,estimated_cost_of_works,rbs_number,commence_by_date,completed_by_date,permit_certificate_type,year
31601,BP-2020-1544,7362369321433/A,1596499200000,Unit 2002 Level 20 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Building Permit,2020
31602,BP-2020-1544,7362369321433/A,1596499200000,Unit 2202 Level 22 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Building Permit,2020
31603,BP-2020-1544,7362369321433/A,1596499200000,Unit 2409 Level 24 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Building Permit,2020
31604,BP-2020-1544,58176/7362369321433/0,1613433600000,Unit 2104 Level 21 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31605,BP-2020-1544,7362369321433/A,1613433600000,Unit 1909 Level 19 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31606,BP-2020-1544,7362369321433/A,1613433600000,Unit 2002 Level 20 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31607,BP-2020-1544,7362369321433/A,1613433600000,Unit 2609 Level 26 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31608,BP-2020-1544,7362369321433/A,1613433600000,Unit 2202 Level 22 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31609,BP-2020-1544,7362369321433/A,1613433600000,Unit 2104 Level 21 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021
31610,BP-2020-1544,7362369321433/A,1613433600000,Unit 2102 Level 21 38 Rose Lane MELBOURNE VIC ...,"Lots 1909, 2002, 2009, 2102, 2104 2109, 2110, ...",487857,BS-15013,2021-08-04,2022-08-04,Certificate of Final Inspection,2021


In [64]:
predictor[predictor['council_ref'] == 'BP-2016-2421']

,council_ref,permit_number,issue_date,address,desc_of_works,estimated_cost_of_works,rbs_number,commence_by_date,completed_by_date,permit_certificate_type,year
14343,BP-2016-2421,2622020160057/0,1478131200000,269-321 Lonsdale Street MELBOURNE VIC 3000,(LG-26) Tenancy fitout of Shop (6c) - Fleet St...,45000,None,2017-11-03,2019-11-03,Building Permit,2016
62171,BP-2016-2421,2622020160057/0,1478131200000,Shop 26 LwrGround 287 Lonsdale Street MELBOURN...,(LG-26) Tenancy fitout of Shop (6c) - Fleet St...,45000,None,2017-11-03,2019-11-03,Building Permit,2016


In [65]:
set(predictor['permit_certificate_type'])

{'Building Permit',
 'Certificate of Final Inspection',
 'Certificate of Occupancy',
 'Occupancy Permit'}

Appears that for a single building, a building permit has to be given out for each room that will be going under construction, given that the building is an apartment. Further, there seems to be overlapping records if the location has multiple options of displaying the address (lower ground, level, unit, etc.). Thus, to reduce a single apartment having too much leverage on the model, we should drop duplicates by permit_number. 

Only keeping 1 record per permit number will resolve the following:
- Slightly differing addresses causing multiple records
- Removal of same apartment having multiple records per building work due to it impacting multiple rooms/units
- It will keep the difference between building permit, certificate of final inspection, and so on. This will give points for more active buildings which have their construction complete


In [147]:
# Remove duplicate permit numbers

predictor_clean = predictor.drop_duplicates(subset=['permit_number'])
len(predictor_clean)

30968

It appears that we have have roughly removed half the records by doing so. Now, we group the dataset by year so that we can get a dataset we can use in our model

In [148]:
predictor_clean = predictor_clean.groupby('year')['council_ref'].count().reset_index(name='count')

Note that our response dataset only has data between 2001 ~ 2021. We remove the year 2000, but 2022 ~ 2024 can be used as the model's performance indicator :D

In [149]:
predictor_clean = predictor_clean[predictor_clean['year'] > 2000]

In [150]:
predictor_clean = predictor_clean.rename(columns={'count': 'building_permit_count'})

Now that we've finalised processing both datasets, we join them

In [194]:
analysis_df = pd.merge(predictor_clean, response_clean, on='year')
analysis_df['year'] = pd.to_numeric(analysis_df['year'])
analysis_df['building_permit_count'] = pd.to_numeric(analysis_df['building_permit_count'])
analysis_df['population'] = pd.to_numeric(analysis_df['population'])


In [198]:
partial_model = ols(
    formula='population ~ building_permit_count',
    data=analysis_df
).fit()

print(partial_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     61.04
Date:                Thu, 16 May 2024   Prob (F-statistic):           2.38e-07
Time:                        12:03:06   Log-Likelihood:                -237.49
No. Observations:                  21   AIC:                             479.0
Df Residuals:                      19   BIC:                             481.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              8.575e+

In [199]:
full_model = ols(
    formula='population ~ building_permit_count + year',
    data=analysis_df
).fit()

print(full_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     429.4
Date:                Thu, 16 May 2024   Prob (F-statistic):           6.47e-16
Time:                        12:03:06   Log-Likelihood:                -211.79
No. Observations:                  21   AIC:                             429.6
Df Residuals:                      18   BIC:                             432.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -1.074e+

It appears that adding the year as a variable after having building permit counts is still statistically significant. We also test for interaction

In [201]:
interaction_model = ols(
    formula='population ~ building_permit_count + year + building_permit_count*year',
    data=analysis_df
).fit()

print(interaction_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     500.4
Date:                Thu, 16 May 2024   Prob (F-statistic):           8.93e-17
Time:                        12:09:38   Log-Likelihood:                -205.43
No. Observations:                  21   AIC:                             418.9
Df Residuals:                      17   BIC:                             423.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

Interaction also seems to be significant, even with the other 2 features already being added to the model. Test for goodness of fit

In [202]:
interaction_model.aic < full_model.aic

True

In [203]:
anova_lm(interaction_model)

,df,sum_sq,mean_sq,F,PR(>F)
building_permit_count,1.0,2.627851e+10,2.627851e+10,1157.817349,4.438442e-17
year,1.0,7.472806e+09,7.472806e+09,329.247827,1.459208e-12
building_permit_count:year,1.0,3.215219e+08,3.215219e+08,14.166084,1.547890e-03
Residual,17.0,3.858422e+08,2.269660e+07,NaN,NaN


Attempt to predict 2022, 2023 and 2024 population

In [207]:
predict_df = predictor_clean.iloc[21:]

In [208]:
interaction_model.predict(predict_df)

22    194064.006211
23    201738.853351
24    175977.057089
dtype: float64

Round the numbers to the nearest whole. Thus, our final conclusion can be:

### 2022

194064 people

### 2023

201739 people

### 2024

175977 people